# Topic Modeling

Author: Crystal

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time
import time

import nltk
import stanza
import gensim
import re

#topic modeling
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression as lm

import matplotlib.collections as mcol
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from matplotlib.lines import Line2D

import seaborn as sns

In [2]:
df = pd.read_csv("/project/class/bii_sdad_dspg/ncses_oss_2021/requests_scrape/oss_readme_aggregated/oss_readme_data_061521.csv")

df.reset_index(inplace = True)

In [3]:
df

,index,slug,readme_text,batch,as_of,status
0,0,h2oota/emacs-win64-msvc,Copyright (C) 2001-2016 Free Software Foundati...,oss_readme_batch1_1,6/11/21 17:21,Done
1,1,Distrotech/mysql-server,MySQL Server 5.6\r\n\r\nThis is a release of M...,oss_readme_batch1_1,6/11/21 17:21,Done
2,2,DaichiUeura/Emacs-for-Windows--xj-,"Copyright (C) 2001, 2002, 2003, 2004, 2005, 20...",oss_readme_batch1_1,6/11/21 17:21,Done
3,3,httpgit12/jb4evea-16,"<img alt=""Swift logo"" height=""70"" src=""https:/...",oss_readme_batch1_1,6/11/21 17:21,Done
4,4,Tehsurfer/hugo-contrarian,# Contrarian website\nThis website is the face...,oss_readme_batch1_1,6/11/21 17:21,Done
...,...,...,...,...,...,...
52067,52067,IonPostglacial/treegers,A RTS game using hexagonal grid maps.\n,oss_readme_batch1_9,2021-06-14 15:06:23,Done
52068,52068,TheSavior/PerfectBut,NaN,oss_readme_batch1_9,2021-06-14 15:06:24,Done
52069,52069,kchivorn/angular2,# Angular QuickStart Source\n[![Build Status][...,oss_readme_batch1_9,2021-06-14 15:06:24,Done
52070,52070,jeffet/vehicle-manufacture-20180318202618486,# Blockchain - Tutorial\n\nThis is the tutoria...,oss_readme_batch1_9,2021-06-14 15:06:23,Done


# Tokenization

In [5]:
# Create a stanza pipeline: this pipeline will tokenize, determine pos, 
# and then lemmatize the token appropriately.

nlp = stanza.Pipeline(lang='en',processors='tokenize,pos,lemma',tokenize_batch_size=500,lemma_batch_size=500,
                      use_gpu = True)

# Note - I thought about keeping the pos PART if the lemma was "not", ex) isn't --> be, not.  But due to the fact
# that we are using a bag-of-words model that does not take into account word order, this won't matter in the
# topic model.  So, this is not kept.


def token_pos_lemma(doc, pretokened=False, keep_numbers=True):
    
    # This function uses the pipeline to tokenize, find POS, and lemmatize a document
    
    """if pretokened, dont use this function, as it hasnt been adapted for it"""
    assert not pretokened #If these are already tokened per another pipeline, this function won't work correctly
    
    new_tokens=[]
     
    processed=nlp(doc)  # this is the line that does the tokenizing, pos, and lemmatizing
    
    for sent in processed.sentences:
        for word in sent.words:
            
            #If its a regular noun, verb, adj, or adverb, keep lemmatized form
            if word.pos in ['NOUN','VERB','ADJ','ADV']:
                new_tokens.append(word.lemma)
            
            #If you decided to retain numbers, their lemma is kept here. 
            #Note that number catching isnt perfect by this lemmatizing.
            elif word.pos=='NUM' and keep_numbers:
                new_tokens.append(word.lemma)
            
            #Exact phrases are kept here with no attempt at lemmatization: e.g. mars does not become mars, 
            #and hopefully scientific words e.g. chemicals will be tagged as propn, x, or intj if needed
            elif word.pos in ['PROPN','X','INTJ']: 
                new_tokens.append(word.text)
            
            #Note that no other tokens are kept        
       
    return new_tokens

token_pos_lemma("a text long hello wow i'm at mia's house studying.")

2021-07-06 21:46:18 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-07-06 21:46:18 INFO: Use device: cpu
2021-07-06 21:46:18 INFO: Loading: tokenize
2021-07-06 21:46:18 INFO: Loading: pos
2021-07-06 21:46:18 INFO: Loading: lemma
2021-07-06 21:46:18 INFO: Done loading processors!


['text', 'long', 'hello', 'wow', "i'm", 'mia', 'house', 'study']

In [6]:
df_try = df.loc[1:1000,:]
df_try

,index,slug,readme_text,batch,as_of,status
1,1,Distrotech/mysql-server,MySQL Server 5.6\r\n\r\nThis is a release of M...,oss_readme_batch1_1,6/11/21 17:21,Done
2,2,DaichiUeura/Emacs-for-Windows--xj-,"Copyright (C) 2001, 2002, 2003, 2004, 2005, 20...",oss_readme_batch1_1,6/11/21 17:21,Done
3,3,httpgit12/jb4evea-16,"<img alt=""Swift logo"" height=""70"" src=""https:/...",oss_readme_batch1_1,6/11/21 17:21,Done
4,4,Tehsurfer/hugo-contrarian,# Contrarian website\nThis website is the face...,oss_readme_batch1_1,6/11/21 17:21,Done
5,5,jrockway/emacs,"Copyright (C) 2001, 2002, 2003, 2004, 2005, 20...",oss_readme_batch1_1,6/11/21 17:21,Done
...,...,...,...,...,...,...
996,996,krg7880/json-schema-generator,json-schema-generator\n=====================\n...,oss_readme_batch1_10,2021-06-14 15:15:29,Done
997,997,mobiusbyte/bumpit,# bumpit\n[![CircleCI](https://circleci.com/gh...,oss_readme_batch1_10,2021-06-14 15:15:29,Done
998,998,vlingo-net/vlingo-net-http,# xoom-net-http\n\n[![Build status](https://ci...,oss_readme_batch1_10,2021-06-14 15:15:29,Done
999,999,weakish/gister,gister\n======\n\n`gister` is a command line t...,oss_readme_batch1_10,2021-06-14 15:15:29,Done


In [7]:
t1 = time.time()
#df_lemma = df_try.assign(readme_lemma= df_try["readme_text"].apply(lambda x: token_pos_lemma(x)))

lemma = []
lemma = df_try["readme_text"].apply(lambda x: token_pos_lemma(x))

t2 = time.time()

print(f"time spent: {t2-t1}")

AssertionError: input should be either str, list or Document

In [ ]:
df_try["readme_lemma"]= lemma

In [37]:
#df_try.to_pickle("/home/zz3hs/git/dspg21oss/data/dspg21oss/readme_lemma_try.pkl")